# 5.5读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## 5.5.1.**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [11]:
import torch
from torch import nn
import torch.nn.functional as F

x = torch.arange(4)
#jupyter文件的默认储存位置
torch.save(x, 'x-file')
#也可以更改存储数据的位置（按照绝对路径进行存储）
torch.save(x,'D:/BaiduNetdiskDownload/x-filess')

torch.save介绍
https://pytorch.org/docs/stable/generated/torch.save.html?highlight=torch+save#torch.save

我们现在可以将存储在文件中的数据读回内存。


In [1]:
x2 = torch.load('x-file')
print(x2)

NameError: name 'torch' is not defined

我们可以[**存储一个张量列表，然后把它们读回内存。**]


#### 列表的简单介绍

https://www.runoob.com/python/python-lists.html

列表是最常用的Python数据类型

列表的数据项不需要具有相同的类型

创建一个列表，只要把逗号分隔的不同的数据项使用方括号括起来即可。如下所示：

In [4]:
#1,创建列表
list1 = ['physics', 'chemistry', 1997, 2000]
list2 = [1, 2, 3, 4, 5,6]
list3 = ["a", "b", "c", "d"]
#2，访问列表
#与张量的访问方式一样，列表索引从0开始。列表可以进行截取、组合
print(list1[0])# list1[0]的值
print(list2[1:5])# list2[1:5]，左闭右开的，访问索引值为1，2，3，4的元素，也就是第2，3，4，5个
#3，更新列表
list = []          ## 空列表
list.append('Google')   ## 使用 .append() 添加元素
list.append('Runoob')
print(list)
#4,删除列表
list1 = ['physics', 'chemistry', 1997, 2000]
print(list1)
del list1[2]#删除列表中指定的元素
print("After deleting value at index 2 : ")
print(list1)

physics
[2, 3, 4, 5]
['Google', 'Runoob']
['physics', 'chemistry', 1997, 2000]
After deleting value at index 2 : 
['physics', 'chemistry', 2000]


In [13]:
y = torch.zeros(4)#长度维4的一维张量
torch.save([x, y],'x-files')#[x,y]列表中的两个元素分别是张量x、张量y
x2, y2 = torch.load('x-files')#从文件中读取数据
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。


In [14]:
mydict = {'x': x, 'y':y}#第一个键为x,对应的值为张量x,第二个键为y对应的值为张量y
torch.save(mydict, 'mydict')#保存字典
mydict2 = torch.load('mydict')#读取字典，并把读取结果赋值给mydiict2
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 5.5.2.**加载和保存模型参数**

保存单个权重向量（或其他张量）确实有用，
**但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处**。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
需要注意的一个重要细节是，这将**保存模型的参数而不是保存整个模型**。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，**为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载已经保存的参数**。
让我们从熟悉的多层感知机开始尝试一下。
## 序列化的简单介绍 
为什么要序列化？

其实序列化最终的目的是为了对象可以跨平台存储，和进行网络传输。而我们进行跨平台存储和网络传输的方式就是IO，而我们的IO支持的数据格式就是字节数组。

因为我们单方面的只把对象转成字节数组还不行，因为没有规则的字节数组我们是没办法把对象的本来面目还原回来的，所以我们必须在把对象转成字节数组的时候就制定一种规则（序列化），那么我们从IO流里面读出数据的时候再以这种规则把对象还原回来（反序列化）。

如果我们要把一栋房子从一个地方运输到另一个地方去，序列化就是我把房子拆成一个个的砖块放到车子里，然后留下一张房子原来结构的图纸，反序列化就是我们把房子运输到了目的地以后，根据图纸把一块块砖头还原成房子原来面目的过程

参考链接：https://zhuanlan.zhihu.com/p/40462507


In [6]:
#包含一个隐藏层的线性神经网络
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)#隐藏层
        self.output = nn.Linear(256, 10)#输出层

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()#实例化网络
X = torch.randn(size=(2, 20))
Y = net(X)#将数据传递给模型
#print(Y)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]

state_dict()是一个函数，返回一个包含整个模型的参数的有序字典,键是参数的名字，对应的值为参数值。

https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module.state_dict

### model.eval()介绍
一般情况下，我们训练过程如下：

1、拿到数据后进行训练，在训练过程中，使用

model.train(）：告诉我们的网络，这个阶段是用来训练的，可以更新参数。

2、训练完成后进行预测，在预测过程中，使用

model.eval() ： 模型中有BatchNormalization和Dropout，在预测时使用model.eval()后会将其关闭以免影响预测结果。因为Dropout只用于训练集，而不用于测试集。也就是说告诉我们的网络，这个阶段是用来测试的。


In [16]:
#保存包含模型参数的状态字典
torch.save(net.state_dict(), 'mlp.params')
print(net.state_dict())#是有序字典，包括了两层线性全连接层的权重和偏执（前面定义的包含一个隐藏层的多层感知机的所有的参数）
#有序字典和通常字典类似，只是它可以记录元素插入其中的顺序，而一般字典是会以任意的顺序迭代的。

OrderedDict([('hidden.weight', tensor([[ 0.1466,  0.1546, -0.1447,  ...,  0.2060,  0.1407,  0.1187],
        [-0.0646,  0.0516,  0.1553,  ..., -0.2193,  0.1691,  0.1678],
        [-0.0721,  0.2059, -0.0427,  ...,  0.0294,  0.1033, -0.1861],
        ...,
        [ 0.0628, -0.1587,  0.1595,  ...,  0.0454,  0.0604, -0.0561],
        [-0.0771, -0.0201,  0.0466,  ...,  0.2197,  0.1743,  0.2098],
        [-0.1619,  0.2094, -0.0399,  ...,  0.0864,  0.1509, -0.0713]])), ('hidden.bias', tensor([ 0.0898,  0.2019, -0.1800,  0.0750,  0.0812, -0.0390,  0.2076, -0.0920,
        -0.1772,  0.1655, -0.0095, -0.0533, -0.0025,  0.1439, -0.0531, -0.1180,
        -0.0167, -0.0317,  0.1658,  0.0756,  0.0631, -0.1290, -0.2110,  0.0101,
        -0.0241,  0.0963,  0.1520, -0.2036, -0.1057, -0.1390,  0.0072, -0.0101,
         0.2015, -0.0925, -0.0650, -0.1406, -0.1050, -0.0717, -0.0053,  0.2027,
        -0.0540,  0.0798, -0.1860, -0.0115, -0.0071,  0.0144, -0.1936, -0.1720,
        -0.2051, -0.0866,  0.0973,  0

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)


In [17]:
clone = MLP()#实例化原始多层感知机的一个备份
clone.load_state_dict(torch.load('mlp.params'))#将前面储存的包含参数的字典加载出来，利用.load_state_dict使得多层感知机clone的参数并不是随机初始化的而是前面我们保存的参数
clone.eval()#从train模式调整为test模式，不再进行训练，不再更改梯度

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。


In [18]:
Y_clone = clone(X)#向clone里面传递数据
Y_clone == Y#Y=net（X)

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

### 5.5.3.保存和加载震整个模型
存整个网络模型:网络结构+权重参数 

In [23]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)#隐藏层
        self.output = nn.Linear(256, 10)#输出层

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()#实例化网络
#保存模型net
torch.save(net,'model')
#读取模型
net_copy=torch.load('model')
#向其传递数据，测试是否实现了模型的读取
net_copy(X)==net(X)

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。



[Discussions](https://discuss.d2l.ai/t/1839)
